In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from benchmarkrewriter.benchmark_parser import WorkerBenchmarkParser
import os

path_to_benchmarks = r'/home/dhu/Downloads/benchmarks'
path_to_results = r'/home/dhu/Downloads/results/'

In [ ]:
benchmarks = os.listidr(path_to_benchmarks)
result_files = os.listdir(path_to_results)


In [ ]:
# TODO: make sure names match
# read benchmarks
benchmark_data = dict()
# read results
results = dict()

In [ ]:
def calculate_value(fitness, best):
    return ((fitness - best) / best)

In [ ]:
def ecdf_inf(vectors, column, max : bool = False, n_instances : int = 402, labels : list[str] = [], instances : list[str] = []):
    plot_vectors = []
    for vector in vectors:
        plot_vectors.append([[0.0],[0.0]])
        i = 1
        while i < len(vector):
            if vector[i] == float('inf'):
                break
            if vector[i] == -float('inf'):
                break
            while i < len(vector) and vector[i] == vector[i-1]:
                i += 1
            plot_vectors[-1][0].append(vector[i-1])
            plot_vectors[-1][1].append((i-1)/n_instances)
            i += 1
    for i in range(len(vectors)):
        plt.plot(plot_vectors[i][0], plot_vectors[i][1], label=[labels[i]])
    if column == 'fitness_value':
        plt.xlim(-0.1, 1.0)
        
    plt.legend()
    plt.title(column)
    plt.show()

In [ ]:
def get_min_max_value(filters, filter):
    max_filter = str('max_'+filter)
    min_filter = str('min_'+filter)
    max_value = float('inf') if max_filter not in filters else filters[max_filter]
    min_value = -float('inf') if min_filter not in filters else filters[min_filter]
    return max_value, min_value

def check(value, max, min):
    return value >= max and value <= min
# NOTE: names of benchmark instances and result instances need to match / be mapped
def get_plot_vectors(results : dict, benchmarks : dict, filters : dict):
    max_dv, min_dv = get_min_max_value(filters, 'dv')
    max_flexibility, min_flexibility = get_min_max_value(filters, 'flexibility')
    max_operations, min_operations = get_min_max_value(filters, 'operations')
    # ...
    plot_data = dict()
    for key in benchmarks.keys():
        if check(benchmarks[key]['dv'], max_dv, min_dv) and check(benchmarks[key]['flexibility'], max_flexibility, min_flexibility) and check(benchmarks[key]['n_operations'], max_operations, min_operations):    
            plot_data[key] = dict()
            best = float('inf')
            for solver in results.keys():
                if key in results[solver] and results[solver][key] < best:
                    best = results[solver][key]
            for solver in results.keys():
                if key in results[solver]:
                    plot_data[key][solver] = calculate_value(results[solver][key], best)
                else:
                    plot_data[key][solver] = float('inf')
    vectors = []
    for solver in results.keys():
        vector = []
        for instance in plot_data.keys():
            vector.append(plot_data[instance][solver])
        vector.sort()
        vectors.append(vector)
    return vectors



In [ ]:
example_filter = {'max_dv': 0.2, 'min_flexibility': 0.3, 'min_operations': 50, 'max_operations': 200}
vectors = get_plot_vectors(results, benchmark_data, example_filter)